In [1]:
import torch
import pandas as pd
import torch_geometric as pyg
import matplotlib.pyplot as plt

确定数据源：首先，你需要收集客户的历史交易数据，这通常包括交易日期、交易金额等信息。

计算Recency (R)：确定每个客户的最近一次购买日期，并计算从该日期到当前日期的时间间隔。时间间隔越短，表明客户最近有交易，其R值越小，通常认为客户价值越高。

计算Frequency (F)：统计每个客户在特定时间段（如一年）内的购买次数。购买次数越多，表明客户忠诚度越高，F值越大。

计算Monetary (M)：计算每个客户在特定时间段内的总消费金额。消费金额越高，表明客户的购买力越强，M值越大。

In [7]:
customers = pd.read_csv('../data/customers_m1.csv')

In [8]:
customers.head()

,客户id,位置(新增),是否医保店,客户等级,客户店员数,是否有过投诉,连锁标记(更新2),注册时长,客户类型_医院,客户类型_单体药店,客户类型_卫生服务站,客户类型_诊所,客户类型_连锁药店
0,7081090359123820544,1.064867,0.972626,1.709349,-0.032001,-0.814386,-0.12702,-1.422691,False,False,False,True,False
1,6970904116784796290,-0.118820,0.972626,1.255499,0.344175,1.227912,-0.12702,1.096914,False,False,True,False,False
2,6970904089375019075,-1.302507,-1.028139,-1.013755,0.720350,1.227912,-0.12702,1.484774,False,False,True,False,False
3,6970903995691045348,-0.118820,0.972626,-1.013755,0.344175,-0.814386,-0.12702,1.102836,False,False,True,False,False
4,6970903992650174547,-0.118820,-1.028139,0.801648,0.720350,-0.814386,-0.12702,-0.262074,False,False,True,False,False


In [4]:
orders = pd.read_csv('../data/orders_2023.csv')

In [5]:
orders.head()

,客户id,订单号,商品编码,数量,商品原价,成交单价,实付金额,折扣金额,应付金额,退款金额,...,活动id,是否活动,活动类型,支付状态,订单创建日期,订单创建时间,订单支付日期,订单支付时间,实际配送时长,Unnamed: 21
0,6970903968834916810,BL2305080755068263791,7049983581464297472,1,19.0,19.0,19.0,0.0,19.0,NaN,...,NaN,否,NaN,已支付,2023-05-06,07:55:07,2023-05-06,07:55:27,3.0,NaN
1,6970903968834916810,BL2305080755068263791,7042381643315150848,2,83.0,83.0,166.0,0.0,166.0,NaN,...,NaN,否,NaN,已支付,2023-05-06,07:55:07,2023-05-06,07:55:27,3.0,NaN
2,6970903968834916810,BL2305080755068263791,7029328967035256832,2,21.5,21.5,43.0,0.0,43.0,NaN,...,NaN,否,NaN,已支付,2023-05-06,07:55:07,2023-05-06,07:55:27,3.0,NaN
3,6970903968834916810,BL2305080755068263791,7028187294771314688,1,22.5,22.5,22.5,0.0,22.5,NaN,...,NaN,否,NaN,已支付,2023-05-06,07:55:07,2023-05-06,07:55:27,3.0,NaN
4,6970903968834916810,BL2305080755068263791,7024974707002834944,2,27.5,27.5,55.0,0.0,55.0,NaN,...,NaN,否,NaN,已支付,2023-05-06,07:55:07,2023-05-06,07:55:27,3.0,NaN


In [9]:
customers['客户id']

0         7081090359123820544
1         6970904116784796290
2         6970904089375019075
3         6970903995691045348
4         6970903992650174547
                 ...         
161081    6970903964061798707
161082    6970903995691045154
161083    7130473638507102281
161084    6993765167095922688
161085    7015469723831812096
Name: 客户id, Length: 161086, dtype: int64

In [10]:
orders.loc[orders['客户id']==customers['客户id'][0]]

,客户id,订单号,商品编码,数量,商品原价,成交单价,实付金额,折扣金额,应付金额,退款金额,...,活动id,是否活动,活动类型,支付状态,订单创建日期,订单创建时间,订单支付日期,订单支付时间,实际配送时长,Unnamed: 21
13364014,7081090359123820544,BL2307110935138036299,6970878582294057377,20,3.29,3.13,62.6,3.2,65.8,NaN,...,7.080479e+18,是,特价,已支付,2023-07-09,09:35:14,2023-07-09,09:35:44,2.0,NaN
13364015,7081090359123820544,BL2307110935138036299,6970878582294057461,10,3.85,3.66,36.6,1.9,38.5,NaN,...,7.080479e+18,是,特价,已支付,2023-07-09,09:35:14,2023-07-09,09:35:44,2.0,NaN
13364016,7081090359123820544,BL2307110935138036299,6970878582294057515,20,5.99,5.69,113.8,6.0,119.8,NaN,...,7.080479e+18,是,特价,已支付,2023-07-09,09:35:14,2023-07-09,09:35:44,2.0,NaN
13364017,7081090359123820544,BL2307110935138036299,6970878582294057646,10,4.69,4.46,44.6,2.3,46.9,NaN,...,7.080479e+18,是,特价,已支付,2023-07-09,09:35:14,2023-07-09,09:35:44,2.0,NaN
13364018,7081090359123820544,BL2307110935138036299,6970878582294057914,10,4.19,3.98,39.8,2.1,41.9,NaN,...,7.080479e+18,是,特价,已支付,2023-07-09,09:35:14,2023-07-09,09:35:44,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22250731,7081090359123820544,BL2312082342441358950,6970878583757869345,10,5.39,5.39,53.9,0.0,53.9,NaN,...,NaN,否,NaN,已支付,2023-12-06,23:42:44,2023-12-06,23:43:20,4.0,NaN
22250732,7081090359123820544,BL2312082342441358950,6988308297589719041,20,7.59,6.35,127.0,24.8,151.8,NaN,...,7.135151e+18,是,特价,已支付,2023-12-06,23:42:44,2023-12-06,23:43:20,4.0,NaN
22250733,7081090359123820544,BL2312082342441358950,6995263451140456448,20,4.69,2.99,59.8,34.0,93.8,NaN,...,7.138436e+18,是,秒杀,已支付,2023-12-06,23:42:44,2023-12-06,23:43:20,4.0,NaN
22250734,7081090359123820544,BL2312082342441358950,7056803927525687296,60,2.20,2.20,132.0,0.0,132.0,NaN,...,NaN,否,NaN,已支付,2023-12-06,23:42:44,2023-12-06,23:43:20,4.0,NaN
